In [ ]:
##Created by Prerit Jain

In [14]:
import importlib
import prompts.router_prompt
importlib.reload(prompts.router_prompt)

<module 'prompts.router_prompt' from 'c:\\Users\\prerit.d.jain\\OneDrive - Accenture\\JTI\\python_codes\\rfx-chatbot-demo\\prompts\\router_prompt.py'>

In [15]:
import os
from data_preprocessing import data_preprocessing
from langchain_ollama import OllamaEmbeddings
from vector_store import vector_store
from uuid import uuid4
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from llm_calling import llm_calling
from prompts.router_prompt import router_prompt

In [ ]:
def file_names(directory):
    files=[]
    for file in os.listdir(directory):
        if file.endswith(".pdf"):
            files.append(os.path.join(directory, file))
    return files

In [ ]:
def langchain_doc_creation(file_path):
    doc=[]
    for f in file_path:
        dp = data_preprocessing(file_path=f)
        doc += dp.load_data()
    return doc

In [ ]:
directory = "./Input_Files"
file_path = file_names(directory)
print(file_path)
doc_input = langchain_doc_creation(file_path)
print(len(doc_input))
embeddings = llm_calling(embedding_model="llama3.2:latest").call_embed_model()
vs_dense = vector_store(collection_name="jti_rfp_dense",embeddings=embeddings,path="./tmp/langchain_qdrant_input").vector_qdrant_dense()
uuids = [str(uuid4()) for _ in range(len(doc_input))]
vs_dense.add_documents(documents=doc_input, ids=uuids)


In [ ]:
directory = "./User_File"
file_path = file_names(directory)
print(file_path)
doc_user = langchain_doc_creation(file_path)
print(len(doc_user))
embeddings = llm_calling(embedding_model="llama3.2:latest").call_embed_model()
vs_dense_user = vector_store(collection_name="jti_rfp_dense_user",embeddings=embeddings,path="./tmp/langchain_qdrant_user").vector_qdrant_dense()
uuids_user = [str(uuid4()) for _ in range(len(doc_user))]
vs_dense_user.add_documents(documents=doc_user, ids=uuids_user)

In [ ]:
retriever_input = vs_dense.as_retriever(search_type="similarity", search_kwargs={"k": 2})
retriever_user = vs_dense_user.as_retriever(search_type="similarity", search_kwargs={"k": 2})

In [16]:
# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["universal vectorstore", "user vectorstore"] = Field(
        ...,
        description="There are two vectorstores. Given a user question choose to route it to the vectorstore for Input Files or a vectorstore for User Files.",
    )

In [17]:
llm = llm_calling(model_name="qwen2.5:7b").call_llm()
structured_llm_router = llm.with_structured_output(RouteQuery)

In [19]:
system = router_prompt
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

question_router = route_prompt | structured_llm_router
print(
    question_router.invoke(
        {"question": "Can you tell me a little about the Evaluation criteria for the RFP?"}
    )
)
print(question_router.invoke({"question": "From the document uploaded, can you tell me about the evaluation criterial for the RFP?"}))

datasource='universal vectorstore'
datasource='user vectorstore'
